# Engineering time series

In [17]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, Isomap, MDS
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
import tsfresh
import umap
from sklearn.ensemble import IsolationForest

# format notebook output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML, Javascript
display(HTML("<style>.container { width:90% !important; }</style>"))

# style pandas display
pd.set_option('display.max_columns', None)

# matplotlib magic
%matplotlib inline

In [3]:
def desc(df):
    display(df.head())
    print(df.shape)
    print(df.dtypes)

# Load data

In [4]:
air_reserve = pd.read_csv('../../data_files/air_reserve.csv')
air_store_info = pd.read_csv('../../data_files/air_store_info.csv', encoding='utf-8')
air_visit_data = pd.read_csv('../../data_files/air_visit_data.csv')
date_info = pd.read_csv('../../data_files/date_info.csv')
hpg_reserve = pd.read_csv('../../data_files/hpg_reserve.csv')
hpg_store_info = pd.read_csv('../../data_files/hpg_store_info.csv')
sample_submission = pd.read_csv('../../data_files/sample_submission.csv')
store_id_relation = pd.read_csv('../../data_files/store_id_relation.csv')

## Merge data

In [5]:
air_reserve['reserve_date'] = air_reserve['reserve_datetime'].apply(lambda x: x[:10])
# air_reserve['reserve_date'] = pd.to_datetime(air_reserve['reserve_date'])

In [6]:
air_reserve['visit_date'] = air_reserve['visit_datetime'].apply(lambda x: x[:10])
# air_reserve['visit_date'] = pd.to_datetime(air_reserve['visit_date'])

In [7]:
air_reserve_daily = air_reserve[[
        'air_store_id',
        'visit_date',
        'reserve_visitors'
    ]].groupby(['air_store_id', 'visit_date'], as_index=False).agg('sum')

In [8]:
desc(air_reserve_daily)

,air_store_id,visit_date,reserve_visitors
0,air_00a91d42b08b08d9,2016-10-31,2
1,air_00a91d42b08b08d9,2016-12-05,9
2,air_00a91d42b08b08d9,2016-12-14,18
3,air_00a91d42b08b08d9,2016-12-17,2
4,air_00a91d42b08b08d9,2016-12-20,4


(29830, 3)
air_store_id        object
visit_date          object
reserve_visitors     int64
dtype: object


In [9]:
air_visit_data = pd.merge(
    left=air_visit_data,
    right=air_reserve_daily,
    on=['air_store_id', 'visit_date'],
    how='left',
    copy=False
)

In [10]:
# should we make a seperate column with binary for nullfill?
air_visit_data['reserve_visitors'].fillna(0, inplace=True)

In [12]:
air_visit_data['visit_date'] = pd.to_datetime(air_visit_data['visit_date'])

In [13]:
desc(air_visit_data)

,air_store_id,visit_date,visitors,reserve_visitors
0,air_ba937bf13d40fb24,2016-01-13,25,0.0
1,air_ba937bf13d40fb24,2016-01-14,32,0.0
2,air_ba937bf13d40fb24,2016-01-15,29,0.0
3,air_ba937bf13d40fb24,2016-01-16,22,0.0
4,air_ba937bf13d40fb24,2016-01-18,6,0.0


(252108, 4)
air_store_id                object
visit_date          datetime64[ns]
visitors                     int64
reserve_visitors           float64
dtype: object


In [34]:
air_visit_data[
    air_visit_data['air_store_id'] == air_visit_data['air_store_id'][0]
][[
    'air_store_id',
    'visit_date',
    'visitors'
]]

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6
5,air_ba937bf13d40fb24,2016-01-19,9
6,air_ba937bf13d40fb24,2016-01-20,31
7,air_ba937bf13d40fb24,2016-01-21,21
8,air_ba937bf13d40fb24,2016-01-22,18
9,air_ba937bf13d40fb24,2016-01-23,26


In [37]:
len(air_visit_data[
    air_visit_data['air_store_id'] == air_visit_data['air_store_id'][0]
][[
    'air_store_id',
    'visit_date',
    'visitors'
]]['visit_date'].unique())

391

In [35]:
tsroll = tsfresh.utilities.dataframe_functions.roll_time_series(
    df_or_dict=air_visit_data[
        air_visit_data['air_store_id'] == air_visit_data['air_store_id'][0]
    ][[
        'air_store_id',
        'visit_date',
        'visitors'
    ]],
    column_id='air_store_id',
    column_sort='visit_date',
    column_kind='visitors',
    rolling_direction=1,
    max_timeshift=None
)

/Users/eric.bunch/recruitVisitorForecasting/kaggleVenv/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:394: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn("Your time stamps are not uniformly sampled, which makes rolling "


In [36]:
tsroll

,visit_date,air_store_id,visitors
1847,2016-01-13,2016-01-13,25
1848,2016-01-14,2016-01-14,32
1849,2016-01-15,2016-01-15,29
1850,2016-01-16,2016-01-16,22
1851,2016-01-18,2016-01-18,6
1852,2016-01-19,2016-01-19,9
1853,2016-01-20,2016-01-20,31
1854,2016-01-21,2016-01-21,21
1855,2016-01-22,2016-01-22,18
1856,2016-01-23,2016-01-23,26
